In [129]:
from datetime import datetime
from dataclasses import dataclass


class Connection:
    def __init__(self, line, departure_time, arrival_time, start_latitude, start_longitude, end_latitude, end_longitude, end_stop):
        self.line: str = line
        self.departure_time: datetime = departure_time
        self.arrival_time: datetime = arrival_time
        self.start_latitude: float = start_latitude
        self.start_longitude: float = start_longitude
        self.end_latitude: float = end_latitude
        self.end_longitude: float = end_longitude
        self.end_stop: str = end_stop
        
    def __repr__(self):
        return f"Connection({self.line}, {self.departure_time.strftime('%H:%M:%S')} -> {self.arrival_time.strftime('%H:%M:%S')}, {self.end_stop})"


    def toDict(self):
        return {
            'line': self.line,
            'departure_time': self.departure_time.strftime('%H:%M:%S'),
            'arrival_time': self.arrival_time.strftime('%H:%M:%S'),
            'start_latitude': self.start_latitude,
            'start_longitude': self.start_longitude,
            'end_latitude': self.end_latitude,
            'end_longitude': self.end_longitude,
            'end_stop': self.end_stop
        }


class BusStop:
    def __init__(self, name: str):
        self.name: str = name
        self.connections: dict[str, list[Connection]] = {}

    def add_connection(self, end_stop: str, connection: Connection):
        if end_stop not in self.connections:
            self.connections[end_stop] = [connection]
        else:
            self.connections[end_stop].append(connection)
    
    def __repr__(self):
        return f"BusStop({self.name}, {len(self.connections)} connections)"
    
    def toDict(self):
        return {
            'name': self.name,
            'connections': {k: [x.toDict() for x in v] for k, v in self.connections.items()}
        }



In [130]:
import pandas as pd
from datetime import datetime

datetime_format = '%Y-%m-%d %H:%M:%S'


def convert_time(time: str) -> datetime:
    hour = int(time[:2])
    if hour >= 24:
        hour -= 24
        return datetime.strptime(f'2025-01-02 {hour:02}{time[2:]}', datetime_format)
    else:
        return datetime.strptime(f'2025-01-01 {time}', datetime_format)


def add_connection(graph: dict[str, BusStop], start_stop: str,
                   end_stop: str, connection: Connection):
    if start_stop not in graph:
        graph[start_stop] = BusStop(name=start_stop)
    if end_stop not in graph:
        graph[end_stop] = BusStop(name=end_stop)
    graph[start_stop].add_connection(end_stop, connection)


def load_csv_data(filename: str):
    df = pd.read_csv(filename)
    graph: dict[str, BusStop] = {}

    for index, row in df.iterrows():
        start_stop = row['start_stop']
        end_stop = row['end_stop']
        departure_time = convert_time(row['departure_time'])
        arrival_time = convert_time(row['arrival_time'])
        start_lat = row['start_stop_lat']
        start_lon = row['start_stop_lon']
        end_lat = row['end_stop_lat']
        end_lon = row['end_stop_lon']
        connection = Connection(
            line=row['line'],
            departure_time=departure_time,
            arrival_time=arrival_time,
            start_latitude=start_lat,
            start_longitude=start_lon,
            end_latitude=end_lat,
            end_longitude=end_lon,
            end_stop=end_stop
        )
        add_connection(graph, start_stop, end_stop, connection)
    
    return graph

In [112]:
graph = load_csv_data('mpk_indexed.csv')

/var/folders/03/n179cc412h58x4qb9mt1bv_m0000gn/T/ipykernel_79092/4144027882.py:26: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [6]:
import pickle

with open('mpk_graph.pickle', 'wb') as f:
    pickle.dump(graph, f)

In [131]:
import pickle

with open('mpk_graph.pickle', 'rb') as f:
    graph = pickle.load(f)

In [132]:
from datetime import timedelta

def find_earliest_connection(connections: list[Connection], current_time: datetime, previous_line=None, transfer_time=2) -> Connection:

    earliest = None
    earliest_departure = None
    
    for conn in connections:
        required_time = current_time
        if previous_line is not None and conn.line != previous_line:
            required_time = current_time + timedelta(minutes=transfer_time)
        
        if conn.departure_time >= required_time and (earliest is None or conn.departure_time < earliest_departure):
            earliest = conn
            earliest_departure = conn.departure_time
    
    return earliest

In [114]:
import heapq
from datetime import timedelta

In [133]:

def dijkstra(start_stop: str, end_stop: str, departure_time: str, graph: dict[str, BusStop], transfer_time: int = 2):

    if start_stop not in graph or end_stop not in graph:
        return None, []

    start_time = convert_time(departure_time)

    distances = {stop: float('inf') for stop in graph}
    previous = {stop: None for stop in graph}
    arrival_times = {stop: None for stop in graph}
    previous_lines = {stop: None for stop in graph}

    distances[start_stop] = 0
    arrival_times[start_stop] = start_time

    priority_queue = [(0, start_stop)]

    visited_nodes = 0
    visited_connections = 0

    while priority_queue:
        current_distance, current_stop = heapq.heappop(priority_queue)

        if current_stop == end_stop:
            break

        if current_distance > distances[current_stop]:
            continue

        visited_nodes += 1 

        current_time = arrival_times[current_stop]
        current_line = previous_lines[current_stop]

        for next_stop, connections in graph[current_stop].connections.items():
            visited_connections += 1  

            earliest_conn = find_earliest_connection(
                connections, 
                current_time, 
                previous_line=current_line, 
                transfer_time=transfer_time
            )

            if earliest_conn is None:
                continue

            travel_time = (earliest_conn.arrival_time - current_time).total_seconds() / 60

            if distances[current_stop] + travel_time < distances[next_stop]:
                distances[next_stop] = distances[current_stop] + travel_time
                previous[next_stop] = (current_stop, earliest_conn)
                arrival_times[next_stop] = earliest_conn.arrival_time
                previous_lines[next_stop] = earliest_conn.line

                heapq.heappush(priority_queue, (distances[next_stop], next_stop))

    print(f"Odwiedzone węzły: {visited_nodes}, odwiedzone krawędzie: {visited_connections}")

    route = []
    current_stop = end_stop
    while current_stop and previous[current_stop]:
        prev_stop, conn = previous[current_stop]
        route.append(conn)
        current_stop = prev_stop

    route.reverse()

    total_time = distances[end_stop] if distances[end_stop] != float('inf') else None
    return total_time, route


def print_path(total_time, route):
    print(f"Czas przejazdu: {total_time} min")
    for conn in route:
        print(f"{conn.line} {conn.departure_time.strftime('%H:%M:%S')} -> {conn.arrival_time.strftime('%H:%M:%S')} {conn.end_stop}")


In [134]:
start = "Iwiny - rondo"
end = "Psie Pole (Rondo Lotników Polskich)"
departure = "11:25:00"

total_time, route = dijkstra(start, end, departure, graph)
print_path(total_time, route)


Odwiedzone węzły: 400, odwiedzone krawędzie: 1209
Czas przejazdu: 58.0 min
110 11:25:00 -> 11:26:00 Vivaldiego
110 11:26:00 -> 11:28:00 Kajdasza
110 11:28:00 -> 11:30:00 Jagodzińska
110 11:30:00 -> 11:31:00 Malinowskiego
110 11:31:00 -> 11:32:00 Konduktorska
110 11:32:00 -> 11:33:00 Buforowa-Rondo
110 11:33:00 -> 11:34:00 BARDZKA (Cmentarz)
110 11:34:00 -> 11:35:00 Morwowa
110 11:35:00 -> 11:36:00 Krynicka
110 11:36:00 -> 11:38:00 Bardzka
110 11:38:00 -> 11:40:00 Kamienna
110 11:40:00 -> 11:41:00 Prudnicka
110 11:41:00 -> 11:42:00 Hubska (Dawida)
16 11:47:00 -> 11:51:00 Kościuszki
16 11:51:00 -> 11:52:00 Komuny Paryskiej
16 11:52:00 -> 11:54:00 pl. Wróblewskiego
16 11:54:00 -> 11:56:00 Urząd Wojewódzki (Impart)
16 11:56:00 -> 11:58:00 most Grunwaldzki
16 11:58:00 -> 12:00:00 PL. GRUNWALDZKI
16 12:00:00 -> 12:02:00 Piastowska
16 12:02:00 -> 12:04:00 Prusa
16 12:04:00 -> 12:05:00 Wyszyńskiego
914 12:07:00 -> 12:08:00 Damrota
914 12:08:00 -> 12:11:00 KROMERA
914 12:11:00 -> 12:13:00 Krome

In [135]:
import heapq
from datetime import datetime
from typing import Callable, List, Dict, Optional, Tuple

def astar(
    start_stop: str, 
    end_stop: str, 
    departure_time: str, 
    graph: dict[str, BusStop], 
    heuristic_function: Callable[[str, str, Dict[str, BusStop]], float],
    transfer_time: int = 2
):

    if start_stop not in graph:
        return None, []
    if end_stop not in graph:
        return None, []

    start_time = convert_time(departure_time)

    distances = {stop: float('inf') for stop in graph}
    previous = {stop: None for stop in graph}
    arrival_times = {stop: None for stop in graph}
    previous_lines = {stop: None for stop in graph}

    distances[start_stop] = 0
    arrival_times[start_stop] = start_time

    priority_queue = [(0 + heuristic_function(start_stop, end_stop, graph), start_stop)]

    visited_nodes = 0
    visited_connections = 0

    while priority_queue:
        print("loop")
        current_f_score, current_stop = heapq.heappop(priority_queue)


        if current_stop == end_stop:
            break

        current_h_score = heuristic_function(current_stop, end_stop, graph)
        current_g_score = current_f_score - current_h_score
        
        if current_g_score > distances[current_stop]:
            continue

        visited_nodes += 1

        current_time = arrival_times[current_stop]
        current_line = previous_lines[current_stop]

        for next_stop, connections in graph[current_stop].connections.items():
            print(next_stop)
            visited_connections += 1


            earliest_conn = find_earliest_connection(
                connections, 
                current_time, 
                previous_line=current_line, 
                transfer_time=transfer_time
            )

            if earliest_conn is None:
                continue

            travel_time = (earliest_conn.arrival_time - current_time).total_seconds() / 60

            change = False
            if current_line and earliest_conn.line != current_line:
                change = True

            if distances[current_stop] + travel_time < distances[next_stop]:
                distances[next_stop] = distances[current_stop] + travel_time
                previous[next_stop] = (current_stop, earliest_conn)
                arrival_times[next_stop] = earliest_conn.arrival_time
                previous_lines[next_stop] = earliest_conn.line

                next_h_score = heuristic_function(next_stop, end_stop, graph, 
                                                  current_stop=current_stop,
                                                  change=change
                                                  )
                heapq.heappush(priority_queue, (distances[next_stop] + next_h_score, next_stop))


    if distances[end_stop] == float('inf') or previous[end_stop] is None:
        print(distances[end_stop] == float('inf'))
        print(previous[end_stop] is None)
        print("Nie znaleziono gowna.")
        return None, []

    route = []
    current_stop = end_stop
    while current_stop and previous[current_stop]:
        prev_stop, conn = previous[current_stop]
        route.append(conn)
        current_stop = prev_stop

    route.reverse()

    total_time = distances[end_stop] if distances[end_stop] != float('inf') else None
    print(f"Odwiedzone węzły: {visited_nodes}, odwiedzone krawędzie: {visited_connections}")
    return total_time, route


In [136]:
def zero_heuristic(next_stop: str, target_stop: str, graph: Dict[str, BusStop], current_stop=None, change=None) -> float:
    return 0

In [137]:
import math

def extract_first_connection(stop_name: str, graph: Dict[str, BusStop]):
    try:
        for next_stop, connections in graph[stop_name].connections.items():
            if connections: 
                return connections[0]
    except (KeyError, IndexError):
        pass
    return None


def euclidean_distance(x1, x2, y1, y2):
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def euclidean_distance_heuristic(next_stop: str, target_stop: str, graph: Dict[str, BusStop],
                                  current_stop=None, change=None) -> float:
    first_next_conn = extract_first_connection(next_stop, graph)
    first_target_conn = extract_first_connection(target_stop, graph)
    
    if first_next_conn is None or first_target_conn is None:
        return 0
    
    dist = euclidean_distance(
        first_next_conn.start_latitude, first_target_conn.start_latitude,
        first_next_conn.start_longitude, first_target_conn.start_longitude
    )
    
    # Konwersja do minut podróży
    # 1 stopień szerokości/długości geograficznej to około 111 km
    # średnia prędkość 30 km/h -> 0.5 km/min
    travel_time_estimate = dist * 111 / 0.4
    
    return travel_time_estimate * 0.2

In [138]:
def euclidean_distance_heuristic_2(next_stop: str, target_stop: str, graph: Dict[str, BusStop],
                                  current_stop=None, change=None) -> float:
    first_next_conn = extract_first_connection(next_stop, graph)
    first_target_conn = extract_first_connection(target_stop, graph)
    
    if first_next_conn is None or first_target_conn is None:
        return 0
    
    dist = euclidean_distance(
        first_next_conn.start_latitude, first_target_conn.start_latitude,
        first_next_conn.start_longitude, first_target_conn.start_longitude
    )
    
    # Konwersja do minut podróży
    # 1 stopień szerokości/długości geograficznej to około 111 km
    # średnia prędkość 30 km/h -> 0.5 km/min
    travel_time_estimate = dist * 111 / 0.4
    
    if change:
        return travel_time_estimate * 0.5
    else:
        return 0

In [139]:
def angle_between_heuristic(next_stop: str, 
                            target_stop: str, 
                            graph: Dict[str, BusStop], 
                            current_stop=None, 
                            change=None
                            ) -> float:
    
    try:
    
        first_next_conn = extract_first_connection(next_stop, graph)
        first_target_conn = extract_first_connection(target_stop, graph)
        first_current_conn = extract_first_connection(current_stop, graph)

        curr_target_dist = euclidean_distance(first_current_conn.start_latitude, first_target_conn.start_latitude,
                                            first_current_conn.start_longitude, first_target_conn.start_longitude)
        
        curr_next_dist = euclidean_distance(first_current_conn.start_latitude, first_next_conn.start_latitude,
                                            first_current_conn.start_longitude, first_next_conn.start_longitude)
        
        next_target_dist = euclidean_distance(first_target_conn.start_latitude, first_next_conn.start_latitude,
                                            first_target_conn.start_longitude, first_next_conn.start_longitude)
        

        cos = (next_target_dist**2- curr_target_dist**2 - curr_next_dist**2) / ( -2.0 * curr_target_dist * curr_next_dist)


        if change and cos < 0.5:
            return (2.5 - cos)
        return 0    
    except:
        return 0
    
                                                                         
    
    


In [140]:
print(graph)

{'Zajezdnia Obornicka': BusStop(Zajezdnia Obornicka, 3 connections), 'Paprotna': BusStop(Paprotna, 3 connections), 'Obornicka (Wołowska)': BusStop(Obornicka (Wołowska), 1 connections), 'Bezpieczna': BusStop(Bezpieczna, 4 connections), 'Bałtycka': BusStop(Bałtycka, 5 connections), 'Broniewskiego': BusStop(Broniewskiego, 5 connections), 'Pola': BusStop(Pola, 2 connections), 'Syrokomli': BusStop(Syrokomli, 3 connections), 'Kasprowicza': BusStop(Kasprowicza, 3 connections), 'pl. Daniłowskiego': BusStop(pl. Daniłowskiego, 3 connections), 'Przybyszewskiego': BusStop(Przybyszewskiego, 1 connections), 'Czajkowskiego': BusStop(Czajkowskiego, 2 connections), 'KOSZAROWA (Uniwersytet)': BusStop(KOSZAROWA (Uniwersytet), 1 connections), 'KOSZAROWA (Szpital)': BusStop(KOSZAROWA (Szpital), 1 connections), 'Berenta': BusStop(Berenta, 3 connections), 'KROMERA': BusStop(KROMERA, 7 connections), 'Mosty Warszawskie': BusStop(Mosty Warszawskie, 3 connections), 'Wyszyńskiego': BusStop(Wyszyńskiego, 4 connect

In [141]:
start = "Iwiny - rondo"
end = "Psie Pole (Rondo Lotników Polskich)"
departure = "12:00:00"

total_time, route = astar(start, end, departure, graph, heuristic_function=zero_heuristic)
# print(total_time)
# print_path(total_time, route)


def serialize_route(route):
    return [(conn.line, conn.departure_time, conn.arrival_time, 
                conn.end_stop, conn.start_latitude, conn.start_longitude) for conn in route]

serialize_route(route)

loop
Vivaldiego
Iwiny - Kolejowa
loop
Kajdasza
Iwiny - rondo
loop
Jagodzińska
Vivaldiego
loop
Malinowskiego
Kajdasza
loop
Konduktorska
Jagodzińska
ROD Zgoda
Lutosławskiego
loop
Buforowa-Rondo
Malinowskiego
loop
BARDZKA (Cmentarz)
Konduktorska
ROD Zgoda
loop
Morwowa
Buforowa-Rondo
GAJ - pętla
loop
Świeradowska
Morwowa
loop
GAJ - pętla
GAJ
Morwowa
loop
Świeradowska
Działkowa
GAJ - pętla
Borowska (Szpital)
loop
GAJ
Śliczna
ROD Bajki
Borowska (Szpital)
loop
Asfaltowa (szkoła)
Malinowskiego
loop
Klasztorna
Lutosławskiego
loop
Świeradowska
Krynicka
Złotostocka
BARDZKA (Cmentarz)
loop
Śliczna
Działkowa
Orzechowa
SPISKA (Ośrodek sportu)
loop
Tunelowa
Konduktorska
loop
Parafialna
Asfaltowa (szkoła)
loop
Morwowa
Bardzka
GAJ - pętla
loop
ROD Mieczyk
ROD Zgoda
loop
Działkowa
Borowska (Aquapark)
ROD Bajki
Uniwersytet Ekonomiczny
loop
Park Brochowski
Tunelowa
loop
Nyska
Kamienna
Krynicka
Orzechowa
loop
Śliczna
DWORZEC AUTOBUSOWY
Uniwersytet Ekonomiczny
Wapienna
PETRUSEWICZA
loop
Wojszyce
Świt
Oboźna

[(145,
  datetime.datetime(2025, 1, 1, 12, 2),
  datetime.datetime(2025, 1, 1, 12, 3),
  'Vivaldiego',
  51.04509727,
  17.06086879),
 (145,
  datetime.datetime(2025, 1, 1, 12, 3),
  datetime.datetime(2025, 1, 1, 12, 5),
  'Kajdasza',
  51.05009367,
  17.05910045),
 (145,
  datetime.datetime(2025, 1, 1, 12, 5),
  datetime.datetime(2025, 1, 1, 12, 7),
  'Jagodzińska',
  51.05422485,
  17.05798773),
 (145,
  datetime.datetime(2025, 1, 1, 12, 7),
  datetime.datetime(2025, 1, 1, 12, 8),
  'Malinowskiego',
  51.05919701,
  17.05650629),
 (145,
  datetime.datetime(2025, 1, 1, 12, 8),
  datetime.datetime(2025, 1, 1, 12, 9),
  'Konduktorska',
  51.06290665,
  17.05554594),
 (145,
  datetime.datetime(2025, 1, 1, 12, 9),
  datetime.datetime(2025, 1, 1, 12, 10),
  'Buforowa-Rondo',
  51.067443,
  17.054504),
 (145,
  datetime.datetime(2025, 1, 1, 12, 10),
  datetime.datetime(2025, 1, 1, 12, 11),
  'BARDZKA (Cmentarz)',
  51.07064457,
  17.05385413),
 (145,
  datetime.datetime(2025, 1, 1, 12, 11),

In [65]:
def initial_solution_nn(graph, start, stops):

    def calculate_distance(stop1, stop2):
        c1 = extract_first_connection(stop1, graph)
        c2 = extract_first_connection(stop2, graph)
        return euclidean_distance(c1.start_latitude, c2.start_latitude, 
                                  c1.start_longitude, c2.start_longitude)
    path = [start]
    unvisited = set(stops) - {start}

    current_stop = start
    while unvisited:
        next_stop = min(unvisited, key=lambda stop: calculate_distance(current_stop, stop))
        path.append(next_stop)
        unvisited.remove(next_stop)
        current_stop = next_stop

    path.append(start)
    cordinates = [(extract_first_connection(x, graph).start_latitude, extract_first_connection(x, graph).start_longitude) for x in path]
    return (path, cordinates)




In [47]:
print(extract_first_connection("Działkowa", graph))

Connection(K, 06:42:00 -> 06:44:00, GAJ)


In [117]:
import folium
from IPython.display import display

path, cords = initial_solution_nn(graph, "Iwiny - rondo", 
                          ["Świeradowska", "PL. GRUNWALDZKI", "Działkowa", "PARK POŁUDNIOWY", 
                           "Psie Pole (Rondo Lotników Polskich)", "KOZANÓW", "Irysowa"])

m = folium.Map(location=cords[0], zoom_start=12)

for i, coord in enumerate(cords):
    folium.Marker(coord, tooltip=f"Przystanek {i+1}").add_to(m)

path

# folium.PolyLine(cords, color="blue", weight=5, opacity=0.7).add_to(m)
# display(m)



['Iwiny - rondo',
 'Świeradowska',
 'Działkowa',
 'PARK POŁUDNIOWY',
 'PL. GRUNWALDZKI',
 'Irysowa',
 'KOZANÓW',
 'Psie Pole (Rondo Lotników Polskich)',
 'Iwiny - rondo']

In [137]:
def evaluate_solution(graph, path, departure_time):
    total_time = 0
    total_route = []

    current_time = datetime.strptime(departure_time, '%H:%M:%S')
    for i in range(len(path) - 1):
        

        connection_time, route = astar(path[i], path[i+1], current_time.strftime('%H:%M:%S'), graph, zero_heuristic)
        total_time += connection_time
        total_route += route
        current_time += timedelta(minutes=connection_time)

    return total_time, total_route
        

path, cords = initial_solution_nn(graph, "Iwiny - rondo", 
                          ["Świeradowska", "PL. GRUNWALDZKI", "Działkowa", "PARK POŁUDNIOWY", 
                           "Psie Pole (Rondo Lotników Polskich)", "KOZANÓW", "Irysowa"])

evaluate_solution(graph, path, "11:25:00")

(255.0,
 [Connection(110, 11:25:00 -> 11:26:00, Vivaldiego),
  Connection(110, 11:26:00 -> 11:28:00, Kajdasza),
  Connection(110, 11:28:00 -> 11:30:00, Jagodzińska),
  Connection(110, 11:30:00 -> 11:31:00, Malinowskiego),
  Connection(110, 11:31:00 -> 11:32:00, Konduktorska),
  Connection(110, 11:32:00 -> 11:33:00, Buforowa-Rondo),
  Connection(110, 11:33:00 -> 11:34:00, BARDZKA (Cmentarz)),
  Connection(110, 11:34:00 -> 11:35:00, Morwowa),
  Connection(21, 11:41:00 -> 11:42:00, Świeradowska),
  Connection(21, 11:42:00 -> 11:44:00, GAJ),
  Connection(145, 11:46:00 -> 11:48:00, Działkowa),
  Connection(145, 11:48:00 -> 11:50:00, ROD Bajki),
  Connection(145, 11:50:00 -> 11:51:00, Śliczna),
  Connection(143, 11:54:00 -> 11:57:00, Uniwersytet Ekonomiczny),
  Connection(9, 12:00:00 -> 12:02:00, Wiśniowa),
  Connection(9, 12:02:00 -> 12:03:00, Jaworowa),
  Connection(9, 12:03:00 -> 12:04:00, Weigla (Szpital)),
  Connection(9, 12:04:00 -> 12:05:00, Pułtuska),
  Connection(9, 12:05:00 -> 12:0

In [ ]:
def generate_neighbourhood(path):
    core_path = path[1:-1]
    neighborhood = set()
    
    for i in range(len(core_path) - 1):
        neighbor = core_path.copy()
        neighbor[i], neighbor[i+1] = neighbor[i+1], neighbor[i]
        neighborhood.add(tuple([path[0]] + neighbor + [path[-1]]))
    
    for i in range(len(core_path)):
        for j in range(len(core_path)):
            if i != j:
                neighbor = core_path.copy()
                city = neighbor.pop(i)
                neighbor.insert(j, city)
                neighborhood.add(tuple([path[0]] + neighbor + [path[-1]]))
    
    for start in range(len(core_path)):
        for end in range(start + 1, len(core_path)):
            neighbor = core_path.copy()
            neighbor[start:end+1] = reversed(neighbor[start:end+1])
            neighborhood.add(tuple([path[0]] + neighbor + [path[-1]]))
    
    return [list(sol) for sol in neighborhood]

generate_neighbourhood(path)

[['Iwiny - rondo',
  'Działkowa',
  'PARK POŁUDNIOWY',
  'PL. GRUNWALDZKI',
  'Irysowa',
  'KOZANÓW',
  'Świeradowska',
  'Psie Pole (Rondo Lotników Polskich)',
  'Iwiny - rondo'],
 ['Iwiny - rondo',
  'Świeradowska',
  'Działkowa',
  'PARK POŁUDNIOWY',
  'PL. GRUNWALDZKI',
  'KOZANÓW',
  'Irysowa',
  'Psie Pole (Rondo Lotników Polskich)',
  'Iwiny - rondo'],
 ['Iwiny - rondo',
  'Działkowa',
  'PARK POŁUDNIOWY',
  'PL. GRUNWALDZKI',
  'Świeradowska',
  'Irysowa',
  'KOZANÓW',
  'Psie Pole (Rondo Lotników Polskich)',
  'Iwiny - rondo'],
 ['Iwiny - rondo',
  'Świeradowska',
  'Działkowa',
  'PL. GRUNWALDZKI',
  'PARK POŁUDNIOWY',
  'Irysowa',
  'KOZANÓW',
  'Psie Pole (Rondo Lotników Polskich)',
  'Iwiny - rondo'],
 ['Iwiny - rondo',
  'Działkowa',
  'PARK POŁUDNIOWY',
  'PL. GRUNWALDZKI',
  'Irysowa',
  'KOZANÓW',
  'Psie Pole (Rondo Lotników Polskich)',
  'Świeradowska',
  'Iwiny - rondo'],
 ['Iwiny - rondo',
  'PL. GRUNWALDZKI',
  'PARK POŁUDNIOWY',
  'Działkowa',
  'Świeradowska',
 

In [139]:
from math import ceil

def generate_neighbourhood(path):
    neighborhood = set()
    mid_index = ceil(len(path) / 2)
    lower_half = path[:mid_index]
    
    for i in range(1, len(lower_half)):
        similar_solution = path[:]
        temp = similar_solution[len(similar_solution) - 1 - i]
        similar_solution[len(similar_solution) - 1 - i] = lower_half[i]
        similar_solution[i] = temp
        neighborhood.add(tuple(similar_solution))
    
    return [list(sol) for sol in neighborhood]

In [146]:


def tabu_search(graph, start, stops, departure_time):
    current_solution, _ = initial_solution_nn(graph, start, stops)
    best_solution = current_solution
    tabu_list = set()
    best_time, best_route = evaluate_solution(graph, best_solution, departure_time)

    k = 0
    while k < 30:
        i = 0
        while i < 10:
            neighbourhood = generate_neighbourhood(current_solution)
            best_neighbour = None
            best_neighbour_time = float('inf')
            best_neighbour_route = []

            neighbourhood = [neighbour for neighbour in neighbourhood if tuple(neighbour) not in tabu_list]

            for neighbour in neighbourhood:
                total_time, total_route = evaluate_solution(graph, neighbour, departure_time)
                if total_time < best_neighbour_time:
                    best_neighbour_time = total_time
                    best_neighbour = neighbour
                    best_neighbour_route = total_route

            if best_neighbour is None:
                break

            tabu_list.add(tuple(best_neighbour))

            if best_neighbour_time < best_time:
                best_solution = best_neighbour
                best_time = best_neighbour_time
                best_route = best_neighbour_route

            current_solution = best_neighbour

            i += 1
        k += 1
    
    return best_solution, best_time, best_route
       

start_station = "Stalowa"
#start_station= "KRZYKI"
stations_string = "most Grunwaldzki;Kochanowskiego;Wiśniowa;PL. JANA PAWŁA II"
#stations_string = "GRABISZYŃSKA (Cmentarz);ZOO;Urząd Wojewódzki (Muzeum Narodowe);most Grunwaldzki;Kochanowskiego;Wiśniowa;PL. JANA PAWŁA II"
#stations_string = "GRABISZYŃSKA (Cmentarz);Fiołkowa;FAT;Hutmen;Bzowa (Centrum Historii Zajezdnia)"
#stations_string = "Kliniki - Politechnika Wrocławska;BISKUPIN;Stalowa;Krucza;rondo Św. Ojca Pio;most Grunwaldzki;SĘPOLNO"
stations_string = "Tarczyński Arena (Lotnicza);Niedźwiedzia;Bujwida;PARK POŁUDNIOWY;Na Niskich Łąkach;BISKUPIN"


stations_string = stations_string.split(";")

tabu_search(graph, start_station, stations_string, "11:25:00")

(['Stalowa',
  'Tarczyński Arena (Lotnicza)',
  'PARK POŁUDNIOWY',
  'Na Niskich Łąkach',
  'Bujwida',
  'BISKUPIN',
  'Niedźwiedzia',
  'Stalowa'],
 206.0,
 [Connection(14, 11:27:00 -> 11:29:00, Pereca),
  Connection(14, 11:29:00 -> 11:30:00, Grabiszyńska),
  Connection(14, 11:30:00 -> 11:31:00, Kolejowa),
  Connection(14, 11:31:00 -> 11:33:00, pl. Legionów),
  Connection(14, 11:33:00 -> 11:35:00, pl. Orląt Lwowskich),
  Connection(14, 11:35:00 -> 11:37:00, PL. JANA PAWŁA II),
  Connection(12, 11:39:00 -> 11:41:00, Młodych Techników),
  Connection(12, 11:41:00 -> 11:42:00, pl. Strzegomski (Muzeum Współczesne)),
  Connection(12, 11:42:00 -> 11:44:00, Wrocław Mikołajów (Zachodnia)),
  Connection(12, 11:44:00 -> 11:46:00, Niedźwiedzia),
  Connection(12, 11:46:00 -> 11:47:00, Małopanewska),
  Connection(12, 11:47:00 -> 11:48:00, Kwiska),
  Connection(20, 11:50:00 -> 11:52:00, DH Astra),
  Connection(20, 11:52:00 -> 11:53:00, Park Zachodni),
  Connection(20, 11:53:00 -> 11:54:00, Bajana),
